# eq2seq 학습을 사용한 숫자 덧셈


**작성자:** [임도형](http://github.com/dhrim)<br>
**원 작성자:** [Smerity](https://twitter.com/Smerity)와 다른이들<br>
**원 문서:** https://keras.io/examples/nlp/addition_rnn/<br>
**작성일:** 2020/08/22<br>
**최종 수정일:** 2020/08/22<br>
**개요:** 숫자 덧셈 문자열의 입력으로 값 문자열을 구한다. 예 "535+61" -> "596"<br>
**원 개요:** A model that learns to add strings of numbers, e.g. "535+61" -> "596".


<br>


# 소개

본 예는 2개의 숫자를 더한 값을 문자열로 출력하는 모델을 학습시킨다.

**예:**
- 입력 : "535+61"
- 출력 : "506"

다음과 같은 다양한 작업에서 보여지듯이 성능을 위해 입력은 선택적으로 연순이 될 수 있다. [Learning to Execute](http://arxiv.org/abs/1410.4615), 
[Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)

이론적으로 순차열 역순은 이런 문제에서 입력과 출력 간에 더 의존성을 더 짧게 한다.

**결과**:<br>
2자리(역순):<br>
- 1 LSTM층(128 유닛), 5K 학습 데이터 = 55 에폭후에 99% 학습/평가 정확도

3자리(역순):<br>
- 1 LSTM층(128 유닛), 50K 학습 데이터 = 100 에폭후에 99% 학습/평가 정확도

4자리(역순):<br>
- 1 LSTM층(128 유닛), 400K 학습 데이터 = 20 에폭후에 99% 학습/평가 정확도

5자리(역순):<br>
- 1 LSTM층(128 유닛), 128K 학습 데이터 = 30 에폭후에 99% 학습/평가 정확도 



# 예제 설명

# Seq2Seq 학습

순차열(sequence)를 입력으로 순차열(sequence)를 출력하는 모델 학습.

보통 인코더 + 디코더의 구조입니다. 인코더는 입력 문자열을 처리하여 문맥(context)를 출력하고 디코더는 이 문맥을 디코딩하여 목적하는 출력을 내도록 학습합니다.


참고 : https://wikidocs.net/24996


<br>

## 학습 내용

다음과 같은 입력문자열에 대하여 출력문자열을 타겟으로 학습한다.

```
입력문자열 --->  출력문자열
'7+81   '  --->  '25  '
'583+2  '  --->  '585 '
'58+12  '  --->  '70  '
'5+85   '  --->  '90  '
```

결과적으로 입력된 문자열에 대한 덧셈을 수행하는 관계를 학습한다.

<br>

## 역순 입력

성능을 위해서 엽력 문자열을 역순으로 해서 사용한다.

```
입력문자열 --->  출력문자열
'   18+7'  --->  '25  '
'  2+385'  --->  '585 '
'  21+85'  --->  '70  '
'   58+5'  --->  '90  '
```

<br>

## 데이터

### 입출력 데이터의 길이
숫자의 최대 길이(DIGIT)가 3이면 2개의 숫자와 덧셈기호의 최대 길이는 7이다. 출력은 최대 4(3자리 숫자의 합은 최대 4자리)이다.


<br>

### one-hot 인코딩
실제 학습에 사용되는 데이터는 각 문자열이 one-hot 인코딩된 값이다.


<br>

### 입력 데이터 모양
입력 문자열의 길이는 7이고 각 문자는 12개 길이로 one-hot 인코딩된다.

사용되는 문자는 0~9까지 숫자 10개와 덧셈기호와 스페이스 해서 총 12개이다. 입력 문자열의 길이는 7이고 1개의 문자는 길이 12의 one-hot 인코딩된 값이라서 1개 데이터의 모양은 (7,12)이다.

입력 문자열이 ' 13+528'이라면 실제 학습에 사용되는 입력값은 다음과 같다.
```
[[1 0 0 0 0 0 0 0 0 0 0 0]    <--- ' '
 [0 0 0 1 0 0 0 0 0 0 0 0]    <--- '1'
 [0 0 0 0 0 1 0 0 0 0 0 0]    <--- '3'
 [0 1 0 0 0 0 0 0 0 0 0 0]    <--- '+'
 [0 0 0 0 0 0 0 1 0 0 0 0]    <--- '5'    
 [0 0 0 0 1 0 0 0 0 0 0 0]    <--- '2'
 [0 0 0 0 0 0 0 0 0 0 1 0]]   <--- '8'
```

<br>

### 출력 데이터 모양
출력 문자열이 '856 '이라면 실제 학습에 사용되는 출력값은 다음과 같다.

```        
[[0 0 0 0 0 0 0 0 0 0 1 0]    <--- '8'
 [0 0 0 0 0 0 0 1 0 0 0 0]    <--- '5'
 [0 0 0 0 0 0 0 0 1 0 0 0]    <--- '6'
 [1 0 0 0 0 0 0 0 0 0 0 0]]   <--- ' '     
```

<br>

## 모델

### 모델 입출력
입력 모양은 (7,12)이고 출력 모양은 (4,12)이다.

7은 입력문자열의 최대 길이, 4는 출력문자열의 최대 길이, 그리고 12는 입출력에 사용되는 문자의 갯수가 12라서 one-hot 인코딩된 값의 길이이다.


<br>

### Question-Answer 모델

Question-Answer 구조의 모델이다. 질문 무자열이 입력되고 이를 encoder가 인코딩한다. 그리고 그 값을 다시  디코더가 decoding해서 답을 구한다.

2개의 LSTM이 각각 인코더, 디코더로 사용된다.

<br>

다음 구조의 모델을 사용한다.
```
model = keras.Sequential()
model.add(LSTM(..., input_shape=(MAXLEN, 12)) # encoder
model.add(RepeatVector(DIGIT+1))
model.add(LSTM(..., return_sequences=True)) # decoder
model.add(Dense(12, activation="softmax"))
```
2개의 LSTM이 있고 첫번째가 인코더이고, 두번째가 디코더이다.


<br>

출력층 Dense의 노드 수가 12개 인것은 12개의 문자중에 1개를 선택하기 위한 것이다. 그래서 activation이 softmax이고.

전체 모델의 출력은 one-hot 인코딩된 4개의 자리값이어야 한다. 이를 위해 4개의 타임 스텝이 필요하다. 이를 위해 디코더 LSTM앞에 RepeatedVector(4)가 있어서 인코딩된 128 차원의 값을 4번 반복한다.

결과적으로 디코더 LSTM은 4개의 값을 출력하고 Dense는 각각 4개의 값에 대한 문자열을 softmax로 출력한다.




## 셋업


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

TRAINING_SIZE = 50000   # 학습 데이터의 갯수
DIGITS = 3              # 덧셈에 사용될 숫자 자리수
REVERSE = True          # 입력 문자열의 역순 여부. 

# 입력 문자열의 최대 길이.
# 숫자 2개의 길이 에 '+' 길이 1을 더한 값이다.
# 숫자 자리수가 3이면 예를 들어 '345+678'과 같이 최대 길이는 7이다.
MAXLEN = DIGITS + 1 + DIGITS


## Generate the data


In [ ]:
# 사용될 문자들. 0~9까지 숫자 10개와 덧셈 '+'와 스페이스 ' '
chars = "0123456789+ "

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:

    # 임의의 숫자를 만드는 함수
    # step1. 최대 자리수를 임의로 선택하고. 최소 1, 최대 3(DIGIT). 선택된 것이 3이라면
    # step2. 각 자리의 숫자를 선택하고. '1', '3', '5'
    # step3. 각 자리의 숫자를 붙여서 문자열 만들고. '135'
    # step4. 만든 문자열을 int로 변환. '135' -> 135
    f = lambda: int( # step4
        "".join( # step3
            np.random.choice(list("0123456789")) # step2
            for i in range(np.random.randint(1, DIGITS + 1))  # step1
        )
    )
    # 2개의 숫자를 선택했다.
    a, b = f(), f()
    # a = 123
    # b = 45

    # 이미 만들었던 문제면 다시 만들자.
    # x+y와 y+x의 중복도 체크하기 위해 소팅해서 체크한다.
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)

    # 문제 문자열을 만들자
    q = "{}+{}".format(a, b)
    # q = '123+45'

    # 문자열 길이가 MAXLEN이 되도록 스페이스를 뒤에 붙인다.
    query = q + " " * (MAXLEN - len(q))
    # query = '123+45 '

    # 정답 문자열을 구한다.
    ans = str(a + b)
    # ans = '168'

    # 정답은 최대 DIGITS + 1 길이이다. 최대 길이가 되도록 스페이스를 앞에 붙인다.
    ans += " " * (DIGITS + 1 - len(ans))
    # ans = ' 168'

    # 역순으로 처리한다면
    # 문제 문자열을 역순으로 한다.
    if REVERSE:
        # '123+45  '가 '  54+321'가 된다.
        query = query[::-1]
        # query = ' 54+321'

    questions.append(query)
    expected.append(ans)

print("Total questions:", len(questions))


Generating data...
Total questions: 50000


# 데이터를 벡터화 하기

인코딩 디코딩을 위한 유틸 클래스 CharacterTable

In [ ]:
class CharacterTable:
    """ 특정 문자열 집합을 가지고:
    + one-hot 인코딩
    + one-hot된 혹은 숫자로 인코딩된 값을 원 무자열로 디코딩
    + softmax 같은 확율 벡터를 원 문자열로 디코딩
    """

    def __init__(self, chars):
        """문자 테이블을 초기화
        # Arguments
            chars: 입력에 사용되는 문자들
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        # self.char_indices = { ' ': 0, 
        #                       '+': 1,
        #                       '0': 2, 
        #                       '1': 3, 
        #                       ...
        #                       '9': 11}
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        # self.indices_char = { 0: ' ', 
        #                       1: '+', 
        #                       2: '0', 
        #                       3: '1', 
        #                       ...
        #                       11: '9'}

    def encode(self, C, num_rows):
        """전달된 문자열 C를 One-hot 인코딩 한다.
        # Arguments
            C: 인코딩할 문자열. 예 ' 123+45'
            num_rows: 반환될 행 수. 입력 길이와 관계없이 반환되는 행 수를 동일하기 위해 사용된다.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C): # 각 자리 문자마다
            x[i, self.char_indices[c]] = 1 # 각 줄의 index만 1로 만든다. one-hot encoding한다.

        # 반환되는 x는 num_rows의 행이고, 각 행은 각 문자에 대한 one-hot 인코딩된 값이다.
        return x

    def decode(self, x, calc_argmax=True):
        """전달된 벡터값 혹은 2D 배열을 해당 문자열로 디코딩한다.
        # Arguments
            x: 벡터 혹은 one-hot 인코딩된 2D 배열 혹은 softmax된 확률 2D 배열 
               혹은 calc_argmax=False일 때는 문자 인덱스 벡터
            calc_argmax: 최대 값의 index 값을 찾을 지 여부. default는 True.
        """

        # one-hot 인코딩 혹은 softmax 값이면 최대 index를 구한다.
        if calc_argmax:
            # x = [ [0 0 0 0 1 0 0 0 0 0 0 0]
            #       [0 0 0 0 0 1 0 0 0 0 0 0]
            #       [0 0 0 0 0 0 1 0 0 0 0 0]]
            x = x.argmax(axis=-1)
            # x = [ 4 5 6 ]
        return "".join(self.indices_char[x] for x in x)


In [ ]:
ctable = CharacterTable(chars)

다음은 인코딩, 디코딩 예이다.

In [ ]:
# 원 문자열
org_str = ' 123+45'
print(f"org_str='{org_str}'")

# 인코딩된 ㄱ밧
encoded = ctable.encode(org_str, MAXLEN)
# [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]      <--- ' '
#  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]      <--- '1'
#  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]      <--- '2'
#  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]      <--- '3'
#  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]      <--- '+'
#  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]      <--- '4'
#  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]     <--- '5'
print(f"encoded=\n{encoded}")
print(f"encoded.shape={encoded.shape}")

# 다시 디코딩해서 원복한 값
decoded = ctable.decode(encoded)
print(f"decoded='{decoded}'")

org_str=' 123+45'
encoded=
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
encoded.shape=(7, 12)
decoded=' 123+45'


In [ ]:
print("Vectorization...")

# DIGITS = 3
# MAXLEN = DIGITS + 1 + DIGITS
# chars = '0123456789+'
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)
# x 데이터 한개의 모양은 (7, 12)  # 7 글자, 12개 글자 종류(0~9, '+', ' ')
# y 데이터 한개의 모양은 (3, 12)  # 3 글자, 12개 글자 종류(0~9, '+', ' ')

# x의 뒷쪽은 대부분 큰 자리수의 문제이다.
# 이를 커버하기 위해 섞는다.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# 명시적으로 10%는 학습에 사용되지 않는 validation 데이터로 딸 둔다.
# 역자주 : 보통 데이터는 train/validation/test로 분리합니다. 
# 학습 시에 사용하지 않는 것은 test 데이터인데 저자가 혼동한 듯 합니다.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)


Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


준비된 데이터는 다음과 같습니다.

In [ ]:
print("x_train[0]\n", x_train[0].astype(np.int))
print()
print("y_train[0]\n", y_train[0].astype(np.int))

x_train[0]
 [[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]]

y_train[0]
 [[0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0]]


준비된 데이터는 인코딩된 값들입니다. 디코딩에서 본 원 문자열은 다음과 같습니다.

In [ ]:
for i in range(10):
  print("'"+ctable.decode(x_train[i])+"' -> '"+ctable.decode(y_train[i])+"'")

' 55+916' -> '674 '
' 491+77' -> '271 '
' 782+26' -> '349 '
'964+035' -> '999 '
' 68+152' -> '337 '
'  2+836' -> '640 '
'  1+249' -> '943 '
' 726+82' -> '655 '
' 08+982' -> '369 '
' 81+104' -> '419 '


## Build the model


# 모델 정의

LSTM을 사용. 모델은 다음과 같다.

```
model.add(layers.LSTM(..., input_shape=(7, 12)) # 7:입력 최대 길이, 12:문자 종류 수
model.add(RepeatVector(4)) # 4:숫자 최대길이 3+1. 4자리가 되도록 값을 반복한다.
model.add(LSTM(..., return_sequences=True)
model.add(Dense(12, activation="softmax")) # 12:문자 종류 수
```

각 층의 출력은 다음과 같다.
```
LSTM            (None, 128)
RepeatVector    (None, 4, 128)    # [1,2] -> [ [1,2], [1,2], [1,2], [1,2] ]
LSTM            (None, 4, 128)
Dense           (None, 4, 12)
```

In [ ]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# 입력 순차열을 LSTM을 사용하여 128길이의 출력으로 인코딩한다.
# 주의 : 입력 길이가 변하는 상황이면 input_shape=(None, num_feature)로 한다.
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))

# 디코더 입력으로 인코딩된 값을 각 시간 스텝(출력 자리) 수 만큼 반복해 준다.
# 출력 최대 길이인 DIGIT+1만큼 반복한다. 
# DIGIT가 3일때 최대 출력값은 999+999=1998이다.
model.add(layers.RepeatVector(DIGITS + 1))

# 디코더 RNN은 한 층 혹은 여러층일 수 있다.
for _ in range(num_layers):
    # return_sequences을 True로 놓아 마지막 스텝뿐 아니라 전체 스텝의 출력값을 
    # (num_samples, timesteps, output_dimc) 모양으로 출력한다.
    # 이것은 다음 층에서 사용되는 TimeDistributed가 입력의 첫 차원이 시간 스텝이어야 하기 때문이다.
    #
    # 역자주 : TensorFlow 2.0이전에는 model.add(TimeDistributed(Dense(..)))의 형태로 쓰여야 했다.
    # 주석에서 TimeDistributed에 관련된 것은 아마도 코드를 업데이트 하면서 주석은 업데이트 하지 않은 것 같다.
    model.add(layers.LSTM(128, return_sequences=True))

# 각 스텝의 출력에 대한 문자열을 선택하도록 한다.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## 모델 학습


In [ ]:
epochs = 30
batch_size = 32

# 학습을 실행하고, 각 학습 중에 validation 데이터에 대한 결과를 보여준다.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )

    # 10개 랜덤 샘플을 골라 결과를 보여준다.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        # rowx = [[[1 0 0 0 0 0 0 0 0 0 0 0]    <--- ' '
        #          [0 0 0 1 0 0 0 0 0 0 0 0]    <--- '1'
        #          [0 0 0 0 0 1 0 0 0 0 0 0]    <--- '3'
        #          [0 1 0 0 0 0 0 0 0 0 0 0]    <--- '+'
        #          [0 0 0 0 0 0 0 1 0 0 0 0]    <--- '5'    
        #          [0 0 0 0 1 0 0 0 0 0 0 0]    <--- '2'
        #          [0 0 0 0 0 0 0 0 0 0 1 0]]]  <--- '8'
        #                                            ' 13+528'
        #
        # rowy = [[[0 0 0 0 0 0 0 0 0 0 1 0]    <--- '8'
        #          [0 0 0 0 0 0 0 1 0 0 0 0]    <--- '5'
        #          [0 0 0 0 0 0 0 0 1 0 0 0]    <--- '6'
        #          [1 0 0 0 0 0 0 0 0 0 0 0]]]  <--- ' '     
        #                                            '856 '   
        preds = np.argmax(model.predict(rowx), axis=-1)
        # preds = [[10  7  8  0]]
        # 요 preds 값은 CharacterTable에 정의된 indices_char의 키 값에 해당한다.
        #   {0: ' ',   # <------
        #    1: '+', 
        #    ...
        #    7: '5'    # <------
        #    8: '6'    # <------
        #    9: '7'
        #    10: '8'   # <------
        #    11: '9'}        

        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        #  q       = ' 13+528'  <--- '825+31 '가 reverse된 것.
        #  correct = '856 '
        #  guess   = '856 '        
        
        # REVERSE = True
        print("Q", q[::-1] if REVERSE else q, end=" ")
        # Q 825+31
        print("T", correct, end=" ")
        # T 856  
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)
        # ☑ 856             



Iteration 1
1407/1407 [==============================] - 13s 5ms/step - loss: 1.7457 - accuracy: 0.3597 - val_loss: 1.5397 - val_accuracy: 0.4214
Q 2+330   T 332  ☒ 334 
Q 5+564   T 569  ☒ 556 
Q 34+709  T 743  ☒ 419 
Q 898+532 T 1430 ☒ 1399
Q 35+62   T 97   ☒ 10  
Q 957+4   T 961  ☒ 966 
Q 918+74  T 992  ☒ 901 
Q 320+35  T 355  ☒ 346 
Q 78+163  T 241  ☒ 119 
Q 534+95  T 629  ☒ 599 

Iteration 2
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3320 - accuracy: 0.4992 - val_loss: 1.1828 - val_accuracy: 0.5497
Q 577+99  T 676  ☒ 631 
Q 889+852 T 1741 ☒ 1732
Q 639+56  T 695  ☒ 691 
Q 550+434 T 984  ☒ 902 
Q 765+24  T 789  ☒ 782 
Q 127+24  T 151  ☒ 132 
Q 71+701  T 772  ☒ 771 
Q 74+89   T 163  ☒ 142 
Q 73+303  T 376  ☒ 338 
Q 85+444  T 529  ☒ 513 

Iteration 3
1407/1407 [==============================] - 7s 5ms/step - loss: 1.0550 - accuracy: 0.6069 - val_loss: 0.9621 - val_accuracy: 0.6463
Q 769+83  T 852  ☒ 856 
Q 29+133  T 162  ☒ 166 
Q 215+814 T 1029 ☒ 1066
Q 869+59 

대략 30 epoch 뒤에 99+% 정도의 validation 정확도를 보인다.